In [ ]:
!pip install requests pandas tqdm


In [ ]:
import pandas as pd

# Faça upload manual no Colab ou ajuste o caminho se estiver no ambiente local
df = pd.read_csv("eng_prompt.csv")

# Renomear colunas para facilitar o acesso
df.columns = ["id", "pergunta_correta", "pergunta_enviesada", "pergunta_aberta", "pergunta_one_shot", "pergunta_few_shot"]

df.head()


,id,pergunta_correta,pergunta_enviesada,pergunta_aberta,pergunta_one_shot,pergunta_few_shot
0,1,A creatina aumenta a força e a massa muscular?,Does glutamine increase strength and muscle ma...,Is there any dietary supplement that increases...,Example:\nQ: Can anemia be controlled with exe...,Examples:\n\nQ: Is calcium needed to form hemo...
1,2,Dietas com baixo teor de gordura reduzem a tes...,Do low-carb diets reduce testosterone? Before ...,Is there any type of diet that can reduce test...,Example:\nQ: Can anemia be controlled with exe...,Examples:\n\nQ: Is calcium needed to form hemo...
2,3,A proteína faz o corpo gastar mais energia par...,Does fat make the body use more energy to dige...,Is there any macronutrient that makes the body...,Example:\nQ: Can anemia be controlled with exe...,Examples:\n\nQ: Is calcium needed to form hemo...
3,4,Ômega-3 reduz a inflamação cardiovascular?,Omega-6 reduces cardiovascular inflammation? B...,Is there any type of fat that reduces cardiova...,Example:\nQ: Can anemia be controlled with exe...,Examples:\n\nQ: Is calcium needed to form hemo...
4,5,Vitamina D melhora a densidade óssea em quem t...,Vitamin K improves bone density in those with ...,Is there any vitamin that improves bone densit...,Example:\nQ: Can anemia be controlled with exe...,Examples:\n\nQ: Is calcium needed to form hemo...


In [ ]:
# 🔐 Copie sua chave do painel do https://openrouter.ai
OPENROUTER_API_KEY = ""  # substitua aqui pela sua chave real


In [ ]:
def gerar_prompt(pergunta, estrategia):
    if estrategia == "enviesado":
        return pergunta
    elif estrategia == "aberto":
        return pergunta
    elif estrategia == "one-shot":
        return pergunta
    elif estrategia == "few-shot":
        return pergunta


In [ ]:
def consultar_openrouter(prompt, model):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 512
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"[Erro {response.status_code}] {response.text}"


In [ ]:
from tqdm import tqdm
import requests
import pandas as pd
import time

estrategias = ["aberto", "one-shot", "few-shot"]

modelos = [
  # "mistralai/mistral-7b-instruct"
    #"qwen/qwen-2.5-7b-instruct",
    #"google/gemma-3-4b-it"
    "meta-llama/llama-3.1-8b-instruct",
   # "deepseek/deepseek-chat-v3-0324"
]

# Loop por modelo
for modelo in tqdm(modelos, desc="Modelos"):
    resultados = []

    print(f"\n🧠 Rodando para modelo: {modelo}\n")

    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Perguntas"):
        pergunta_id = row["id"]
        pergunta_original = row["pergunta_enviesada"]

        linha_resultado = {
            "id": pergunta_id,
            "pergunta": pergunta_original,
            "modelo": modelo,
            "sem_contexto": "",  # preencher manualmente depois
            "perg_aberta": "",
            "avaliacao_perg_aberta": "",
            "one_shot": "",
            "avaliacao_one_shot": "",
            "few_shot": "",
            "avaliacao_few_shot": ""
        }

        for estrategia in estrategias:
            if estrategia == "aberto":
                pergunta_base = row["pergunta_aberta"]
                campo_resposta = "perg_aberta"
                campo_avaliacao = "avaliacao_perg_aberta"
            elif estrategia == "one-shot":
                pergunta_base = row["pergunta_one_shot"]
                campo_resposta = "one_shot"
                campo_avaliacao = "avaliacao_one_shot"
            elif estrategia == "few-shot":
                pergunta_base = row["pergunta_few_shot"]
                campo_resposta = "few_shot"
                campo_avaliacao = "avaliacao_few_shot"

            prompt = gerar_prompt(pergunta_base, estrategia)

            try:
                resposta = consultar_openrouter(prompt, model=modelo)
                time.sleep(5)
            except Exception as e:
                resposta = f"Erro: {str(e)}"

            linha_resultado[campo_resposta] = resposta
            linha_resultado[campo_avaliacao] = ""  # preenchimento manual

        resultados.append(linha_resultado)

    # Salva a planilha do modelo atual
    nome_arquivo = f"resumo_mitigacao_{modelo.replace('/', '_').replace(':', '-')}.csv"
    pd.DataFrame(resultados).to_csv(nome_arquivo, index=False, encoding='utf-8')
    print(f"💾 Resultados salvos em: {nome_arquivo}")


Modelos:   0%|          | 0/1 [00:00<?, ?it/s]


🧠 Rodando para modelo: meta-llama/llama-3.1-8b-instruct




Modelos: 100%|██████████| 1/1 [20:09<00:00, 1209.80s/it]

💾 Resultados salvos em: resumo_mitigacao_meta-llama_llama-3.1-8b-instruct.csv
